<a href="https://colab.research.google.com/github/wenqiglantz/nvidia-sec-finetuning/blob/main/nvidia_sec_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NVIDIA SEC 10-K Filing, Fine-Tuning gpt-3.5-turbo

In [1]:
!pip install llama_index==0.8.19 pypdf sentence-transformers ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.1/745.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.6/272.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.8 MB/s eta 0:00:00

In [2]:
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    Response
)
from llama_index.evaluation import (
    DatasetGenerator,
    QueryResponseEvaluator,
    ResponseEvaluator
)
from llama_index.llms import OpenAI
import pandas as pd
import openai
import os

In [5]:
os.environ["OPENAI_API_KEY"] = "sk-##########"
openai.api_key = os.environ["OPENAI_API_KEY"]

#define LLM
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

In [6]:
!curl https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/4e9abe7b-fdc7-4cd2-8487-dc3a99f30e98.pdf --output nvidia-sec-10k-2022.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1541k  100 1541k    0     0  5751k      0 --:--:-- --:--:-- --:--:-- 5772k


In [7]:
# Shuffle the documents
import random

# load documents
documents = SimpleDirectoryReader(input_files=["nvidia-sec-10k-2022.pdf"]).load_data()
print(f"loaded documents with {len(documents)} pages")

random.seed(42)
random.shuffle(documents)

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

loaded documents with 169 pages


## Generate datasets

Let's first generate two datasets, one for training, the other for eval

### Training dataset

In [9]:
import random
random.seed(42)

questions = []
if os.path.exists("train_questions.txt"):
    with open("train_questions.txt", "r") as f:
        for line in f:
            questions.append(line.strip())
else:
    question_gen_query = (
        "You are a Teacher/ Professor. Your task is to setup "
        "a quiz/examination. Using the provided context from the NVIDIA SEC 10-K filing, formulate "
        "a single question that captures an important fact from the context. "
        "context. Restrict the question to the context information provided."
    )
    # generate questions
    dataset_generator = DatasetGenerator.from_documents(
        documents[:50],
        question_gen_query=question_gen_query,
        service_context=gpt_35_context,
    )

    questions = dataset_generator.generate_questions_from_nodes(num=40)
    print(f"Generated {len(questions)} questions.")

    # save the questions!
    with open("train_questions.txt", "w") as f:
        for question in questions:
            f.write(f"{question.strip()}\n")

In [10]:
for i, question in enumerate(questions, start=1):
    print(f"{i}. {question}")

1. What factors could result in less demand for existing products?
2. How could new or unexpected end use cases impact demand for products?
3. What potential competitive actions could increase demand for competitive products?
4. How could business decisions made by third parties affect product demand?
5. What is the potential impact of increased demand for accelerated or AI-related cloud services on product demand?
6. How could the demand for cryptocurrency mining impact product demand?
7. What potential government actions or changes in governmental policies could affect product demand?
8. What factors have contributed to the significant growth of supply, including inventory on hand, purchase obligations, and prepaid supply agreements?
9. What potential risks are associated with misalignment between inventory or supply commitments and product demand?
10. How have product transitions negatively impacted revenue in the past, and what challenges are associated with shipping new and legacy

### Eval dataset

In [11]:
questions = []
if os.path.exists("eval_questions.txt"):
    with open("eval_questions.txt", "r") as f:
        for line in f:
            questions.append(line.strip())
else:
    dataset_generator = DatasetGenerator.from_documents(
        documents[
            50:
        ],  # since we generated ~1 question for 40 documents, we can skip the first 40
        question_gen_query=question_gen_query,
        service_context=gpt_35_context,
    )

    questions = dataset_generator.generate_questions_from_nodes(num=40)
    print(f"Generated {len(questions)} questions.")

    # save the questions!
    with open("eval_questions.txt", "w") as f:
        for question in questions:
            f.write(f"{question.strip()}\n")

Generated 40 questions.


In [12]:
    for i, question in enumerate(questions, start=1):
        print(f"{i}. {question}")

1. How does the company recognize the benefit from a tax position?
2. What is the company's policy regarding interest and penalties related to unrecognized tax benefits?
3. How is basic net income per share computed?
4. How is diluted net income per share computed?
5. How does the company classify cash equivalents and marketable securities?
6. What is the classification of the company's investments based on their nature and availability for use in current operations?
7. How does the company report available-for-sale debt securities?
8. How are realized gains and losses on the sale of marketable securities recorded?
9. What is the periodic impairment review process for available-for-sale debt investments?
10. How does the company determine the fair value of marketable securities?
11. How are derivative instruments recognized and measured?
12. What is the accounting treatment for changes in the fair value of derivative instruments?
13. What financial instruments potentially subject the c

## Baseline eval for gpt-3.5-turbo

Let's evaluate our base model with both ragas framework and evaluation module


### Eval with ragas

In [41]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

from llama_index import VectorStoreIndex

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

# build vector index and query engine
index = VectorStoreIndex.from_documents(documents, service_context=gpt_35_context)
query_engine = index.as_query_engine(similarity_top_k=2)


In [42]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [43]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)

evaluating with [answer_relevancy]


100%|██████████| 3/3 [00:46<00:00, 15.37s/it]


evaluating with [faithfulness]


100%|██████████| 3/3 [04:13<00:00, 84.44s/it]


{'ragas_score': 0.8947, 'answer_relevancy': 0.9627, 'faithfulness': 0.8356}


In [44]:
import pandas as pd

pd.set_option('display.max_colwidth', 200)
result.to_pandas()

,question,answer,contexts,answer_relevancy,faithfulness
0,How does the company recognize the benefit from a tax position?,The company recognizes the benefit from a tax position by including it as a component of income tax expense. They also accrue for the payment of interest and penalties related to unrecognized tax ...,"[Table of Contents\nprice basis by maximizing the use of observable inputs to determine the standalone selling price for each performance obligation); and (5)\nrecognition of revenue when, or as, ...",0.863815,1.000000
1,What is the company's policy regarding interest and penalties related to unrecognized tax benefits?,The company's policy is to include interest and penalties related to unrecognized tax benefits as a component of income tax expense.,[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nbe subject to limitations due to ownership changes and other limitations provi...,1.000000,1.000000
2,How is basic net income per share computed?,Basic net income per share is computed by dividing the net income by the basic weighted average shares.,[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nNote 5 - Net Income Per Share\nThe following is a reconciliation of the denomi...,0.989098,1.000000
3,How is diluted net income per share computed?,"Diluted net income per share is computed using the weighted average number of common and potentially dilutive shares outstanding during the period, using the treasury stock method. Under the treas...",[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nNote 5 - Net Income Per Share\nThe following is a reconciliation of the denomi...,0.962703,0.666667
4,How does the company classify cash equivalents and marketable securities?,The company classifies cash equivalents as highly liquid investments that are readily convertible into cash and have an original maturity of three months or less at the time of purchase. Marketabl...,[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nWe recognize the benefit from a tax position only if it is more-likely-than-no...,1.000000,0.500000
5,What is the classification of the company's investments based on their nature and availability for use in current operations?,The company classifies its investments as current based on their nature and availability for use in current operations.,[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nidentified for specific customers and an amount based on overall estimated exp...,0.925417,1.000000
6,How does the company report available-for-sale debt securities?,"The company reports available-for-sale debt securities at fair value. The unrealized gains and losses related to these securities are included in accumulated other comprehensive income or loss, wh...",[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nWe recognize the benefit from a tax position only if it is more-likely-than-no...,0.940945,1.000000
7,How are realized gains and losses on the sale of marketable securities recorded?,"Realized gains and losses on the sale of marketable securities are recorded in the other income (expense), net, section of the Consolidated Statements of Income.",[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nThe following tables provide the breakdown of unrealized losses as of January ...,0.973576,1.000000
8,What is the periodic impairment review process for available-for-sale debt investments?,The periodic impairment review process for available-for-sale debt investments involves assessing whether there are any indicators of potential impairment. This can be 

### Eval with evaluation module


In [45]:
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()

def evaluate_query_engine(evaluator, query_engine, questions):
    async def run_query(query_engine, q):
        try:
            return await query_engine.aquery(q)
        except:
            return Response(response="Error, query failed.")

    total_correct = 0
    all_results = []
    for batch_size in range(0, len(questions), 5):
        batch_qs = questions[batch_size:batch_size+5]

        tasks = [run_query(query_engine, q) for q in batch_qs]
        responses = asyncio.run(asyncio.gather(*tasks))
        print(f"finished batch {(batch_size // 5) + 1} out of {len(questions) // 5}")

        # eval for hallucination
        if isinstance(evaluator, ResponseEvaluator):
          for response in responses:
              eval_result = 1 if "YES" in evaluator.evaluate(response) else 0
              total_correct += eval_result
              all_results.append(eval_result)
        # eval for answer quality
        elif isinstance(evaluator, QueryResponseEvaluator):
          for question, response in zip(batch_qs, responses):
              eval_result = 1 if "YES" in evaluator.evaluate(question, response) else 0
              total_correct += eval_result
              all_results.append(eval_result)

        # helps avoid rate limits
        time.sleep(1)

    return total_correct, all_results

In [19]:
# use gpt-4 to evaluate
gpt4_service_context = ServiceContext.from_defaults(llm=OpenAI(temperature=0, llm="gpt-4"))

questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [20]:
# eval for hallucination
evaluator = ResponseEvaluator(service_context=gpt4_service_context)
total_correct, all_results = evaluate_query_engine(evaluator, query_engine, questions)
print(f"Hallucination? Scored {total_correct} out of {len(questions)} questions correctly.")

finished batch 1 out of 8
finished batch 2 out of 8
finished batch 3 out of 8
finished batch 4 out of 8
finished batch 5 out of 8


finished batch 6 out of 8


finished batch 7 out of 8


finished batch 8 out of 8
Hallucination? Scored 18 out of 40 questions correctly.


In [21]:
# eval for answer quality
evaluator = QueryResponseEvaluator(service_context=gpt4_service_context)
total_correct, all_results = evaluate_query_engine(evaluator, query_engine, questions)
print(f"Response satisfies the query? Scored {total_correct} out of {len(questions)} questions correctly.")

finished batch 1 out of 8
finished batch 2 out of 8
finished batch 3 out of 8
finished batch 4 out of 8
finished batch 5 out of 8
finished batch 6 out of 8
finished batch 7 out of 8
finished batch 8 out of 8
Response satisfies the query? Scored 23 out of 40 questions correctly.


## GPT4 to collect training data

In [22]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

In [23]:
questions = []
with open("train_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [24]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=gpt_4_context)
query_engine = index.as_query_engine(similarity_top_k=2)

for question in questions:
    response = query_engine.query(question)

In [25]:
finetuning_handler.save_finetuning_events("finetuning_events.jsonl")

Wrote 65 examples to finetuning_events.jsonl


## Create OpenAIFinetuneEngine

In [26]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "finetuning_events.jsonl"
)

In [27]:
finetune_engine.finetune()

Num examples: 65
First example:
{'role': 'system', 'content': "You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines."}
{'role': 'user', 'content': "Context information is below.\n---------------------\npage_label: 19\nfile_name: nvidia-sec-10k-2022.pdf\n\nTable of Contents\n•new product introductions and transitions resulting in less demand for existing products;\n•new or unexpected end use cases;\n•increase in demand for competitive products, including competitive actions;\n•business decisions made by third parties;\n•the demand for accelerated or AI-related cloud services, including our own software and AI cloud service offerings;\n•the demand for cryptocurrency mining; or\n•government 

In [28]:
finetune_engine.get_current_job()

<FineTuningJob fine_tuning.job id=ftjob-nLjdTv91ozIvnaJFMpVp6lDa at 0x7877a59998f0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-nLjdTv91ozIvnaJFMpVp6lDa",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693793535,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-5ytHcLCFlcB1xR8qyZkYwLRd",
  "result_files": [],
  "status": "running",
  "validation_file": null,
  "training_file": "file-J0qBEx0ka42fL3dwIzWCgc2r",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [29]:
ft_llm = finetune_engine.get_finetuned_model(temperature=0.3)

## Evaluation for fine-tuned model

### Eval with ragas

In [46]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

ft_context = ServiceContext.from_defaults(
    llm=ft_llm
)

In [47]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [48]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=ft_context)

query_engine = index.as_query_engine(similarity_top_k=2)

In [49]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [50]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)

evaluating with [answer_relevancy]


100%|██████████| 3/3 [00:39<00:00, 13.25s/it]


evaluating with [faithfulness]


100%|██████████| 3/3 [03:56<00:00, 78.95s/it]


{'ragas_score': 0.9207, 'answer_relevancy': 0.9596, 'faithfulness': 0.8847}


In [51]:
import pandas as pd

pd.set_option('display.max_colwidth', 200)
result.to_pandas()

,question,answer,contexts,answer_relevancy,faithfulness
0,How does the company recognize the benefit from a tax position?,The company recognizes the benefit from a tax position by classifying it as a current liability or amount refundable if they anticipate payment or receipt of cash for income taxes within one year....,"[Table of Contents\nprice basis by maximizing the use of observable inputs to determine the standalone selling price for each performance obligation); and (5)\nrecognition of revenue when, or as, ...",0.927000,1.000000
1,What is the company's policy regarding interest and penalties related to unrecognized tax benefits?,The company's policy is to include interest and penalties related to unrecognized tax benefits as a component of income tax expense.,[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nbe subject to limitations due to ownership changes and other limitations provi...,0.999999,1.000000
2,How is basic net income per share computed?,Basic net income per share is computed by dividing the net income by the basic weighted average shares.,[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nNote 5 - Net Income Per Share\nThe following is a reconciliation of the denomi...,0.989098,1.000000
3,How is diluted net income per share computed?,"Diluted net income per share is computed using the weighted average number of common and potentially dilutive shares outstanding during the period, using the treasury stock method. Under the treas...",[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nNote 5 - Net Income Per Share\nThe following is a reconciliation of the denomi...,0.962703,0.666667
4,How does the company classify cash equivalents and marketable securities?,The company classifies cash equivalents as highly liquid investments that are readily convertible into cash and have an original maturity of three months or less at the time of purchase. Marketabl...,[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nWe recognize the benefit from a tax position only if it is more-likely-than-no...,0.996730,0.666667
5,What is the classification of the company's investments based on their nature and availability for use in current operations?,The company classifies its investments as current based on the nature of the investments and their availability for use in current operations.,[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nidentified for specific customers and an amount based on overall estimated exp...,0.925411,1.000000
6,How does the company report available-for-sale debt securities?,"The company reports available-for-sale debt securities at fair value. The related unrealized gains and losses are included in accumulated other comprehensive income or loss, which is a component o...",[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nWe recognize the benefit from a tax position only if it is more-likely-than-no...,0.944217,0.800000
7,How are realized gains and losses on the sale of marketable securities recorded?,"Realized gains and losses on the sale of marketable securities are determined using the specific-identification method and recorded in the other income (expense), net, section of the company's Con...",[Table of Contents\nNVIDIA CORPORATION AND SUBSIDIARIES\nNOTES TO THE CONSOLIDATED FINANCIAL STATEMENTS\n(Continued)\nThe following tables provide the breakdown of unrealized losses as of January ...,0.965344,0.500000
8,What is the periodic impairment review process for available-for-sale debt investments?,The context does not provide information on the periodic impairment review process for available-for-sale d

### Eval with evaluation module

In [35]:
# eval for hallucination
evaluator = ResponseEvaluator(service_context=gpt4_service_context)
total_correct, all_results = evaluate_query_engine(evaluator, query_engine, questions)
print(f"Hallucination? Scored {total_correct} out of {len(questions)} questions correctly.")

finished batch 1 out of 8
finished batch 2 out of 8
finished batch 3 out of 8
finished batch 4 out of 8
finished batch 5 out of 8
finished batch 6 out of 8
finished batch 7 out of 8
finished batch 8 out of 8
Hallucination? Scored 17 out of 40 questions correctly.


In [36]:
# eval for answer quality
evaluator = QueryResponseEvaluator(service_context=gpt4_service_context)
total_correct, all_results = evaluate_query_engine(evaluator, query_engine, questions)
print(f"Response satisfies the query? Scored {total_correct} out of {len(questions)} questions correctly.")

finished batch 1 out of 8
finished batch 2 out of 8
finished batch 3 out of 8
finished batch 4 out of 8
finished batch 5 out of 8
finished batch 6 out of 8


finished batch 7 out of 8
finished batch 8 out of 8
Response satisfies the query? Scored 21 out of 40 questions correctly.


## Exploring difference

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

In [ ]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [ ]:
print(questions[0])

How does the company recognize the benefit from a tax position?


### Baseline model

In [ ]:
from llama_index.response.notebook_utils import display_response
from llama_index import ServiceContext
from llama_index.llms import OpenAI


gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

In [ ]:
query_engine = index.as_query_engine(service_context=gpt_35_context)

response = query_engine.query(questions[0])

display_response(response)

**`Final Response:`** The company recognizes the benefit from a tax position by including it as a component of income tax expense. They also accrue for the payment of interest and penalties related to unrecognized tax benefits. However, it is important to note that the amounts asserted by tax authorities could be greater or less than the company's accrued position. As a result, the provisions on tax-related matters may change in the future as revised estimates are made or the underlying matters are settled. Additionally, as of January 29, 2023, the company has not identified any positions for which it is reasonably possible that the total amounts of unrecognized tax benefits will significantly increase or decrease within the next twelve months.

### Fine-tuned model

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI


ft_context = ServiceContext.from_defaults(
    llm=ft_llm
)

In [ ]:
query_engine = index.as_query_engine(service_context=ft_context)

response = query_engine.query(questions[0])

display_response(response)

**`Final Response:`** The company acknowledges the advantage of a tax position by incorporating it into their income tax expense. They also categorize an unacknowledged tax benefit as a current liability or a refundable amount if they expect to pay or receive cash for income taxes within a year. If the payment or receipt of cash for income taxes is projected to occur beyond a year, the amount is classified as a long-term liability or a reduction of a long-term refundable amount.